In [ ]:
from brian2 import *
import csv
import random
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from scipy.spatial import distance

In [ ]:
# Interesting links:

# http://www.maths.dit.ie/~johnbutler/Izhikevich/IzhikevichModel.html#Thalamo-cortical
# https://www.izhikevich.org/publications/spikes.pdf
# https://github.com/brian-team/brian2/issues/809
# http://neuralensemble.org/docs/PyNN/reference/neuronmodels.html

In [ ]:
min_range_V = 5
max_range_V = 10

In [ ]:
list_attack_generation = ["FLO", "SCA"]

### Methods for csv export

In [ ]:
def append_to_csv_file(filename, line):
    # Update the attack file
    with open(filename, 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=';')
        writer.writerow(line)

def dump_simulation_data_to_csv(attack, test, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, vIncrement, paramI, monitor, filename):
    mon_trains = monitor.spike_trains()
    
    for neuron in mon_trains.keys():
        for time_delta in mon_trains[neuron]:
            append_to_csv_file(filename, [attack, test, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, str(vIncrement), str(paramI), round(time_delta/ms, 1), neuron])

def list_neurons_to_string(list_neurons):
    result = ""
    for neuron in list_neurons:
        result += str(neuron)+"-"
        
    return result[:-1]

### General variables and constants

In [ ]:
list_coordinates_optimal_path = [[0,0], [1,0], [1,1], [1,2], [0,2], [0,3], [0,4], [0,5], [1,5], [2,5], 
                                 [2,4], [2,3], [3,3], [3,2], [3,1], [3,0], [4,0], [5,0], [6,0], [6,1], 
                                 [6,2], [5,2], [5,3], [5,4], [5,5], [6,5], [6,6]]

In [ ]:
STEP_TIME = 12*ms
#STEP_TIME = 1000*ms

# STEP_TIME per movement. As there are 27 positions -> 27*STEP_TIME. 
SIMULATION_TIME = (STEP_TIME/ms*len(list_coordinates_optimal_path))*ms

In [ ]:
SIMULATION_TIME

In [ ]:
# Duration of simulation, after STEP_TIME. Divided by the number of neurons
def get_time_steps_sequential(tAttack):
    return trunc(((SIMULATION_TIME-tAttack)/200)/ms)*ms

def get_number_attacks_per_position_sequential(tAttack):
    return STEP_TIME/get_time_steps_sequential(tAttack)

def get_last_instant_attack_sequential(tAttack):
    return tAttack+get_time_steps_sequential(tAttack)*200

In [ ]:
get_time_steps_sequential(50*ms)

In [ ]:
get_number_attacks_per_position_sequential(50*ms)

In [ ]:
get_last_instant_attack_sequential(50*ms)

### Auxiliary methods to translate between coordinates and indexes

In [ ]:
def generate_list_random_neurons(nNeurons):
    list_neurons = list(range(0, 200))
    result = []

    for i in range(0, nNeurons):
        index = randint(0, len(list_neurons))
        
        result.append(list_neurons[index])
        del list_neurons[index]
    
    return sorted(result)

#def neurons_list_to_string(listNeurons):

#    printString = "["
#    for item in sorted(listNeurons):
#        printString+=str(item)+","

#    return printString[:-1]+"]"

In [ ]:
# Dict that stores the index value of a neuron: 3D coordinate -> numeric index (0-199)
dict_neurons_to_numbers = {}

counter = 0

for i in range(0, 5):
    for j in range(0, 5): 
        for k in range(0, 8): 
            dict_neurons_to_numbers[(i,j,k)] = counter
            counter+=1

In [ ]:
dict_neurons_to_numbers

In [ ]:
# Dict that stores the instant in which the mouse is in eah position of the optimal path

dict_instant_optimal_path = {}

counter = 0

for coord in list_coordinates_optimal_path:
    dict_instant_optimal_path[(coord[0],coord[1])]= counter
    counter += STEP_TIME/ms     

# List of the instants previously calculated (list values of dict)
list_values = list(dict_instant_optimal_path.values())
list_instant_optimal_path = []

for value in list_values:
    list_instant_optimal_path.append(round(value))

In [ ]:
def get_visible_coordinates_by_position(x,y):
    
    coords = [[x-1,y-1], [x-1, y], [x-1,y+1], [x,y-1], [x,y], [x,y+1], [x+1,y-1], [x+1,y], [x+1,y+1]]
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 6) and (0 <= n[1] <= 6):
            result.append(n)
            
    return result

In [ ]:
get_visible_coordinates_by_position(3,3)

In [ ]:
# Get the neurons indexes that are related to a given maze coordinate
def get_neurons_indexes_by_position(x, y):
    
    coords = [
        [x-2,y-2,0], [x-2,y-2,1], [x-2,y-2,2], [x-2,y-2,3], [x-2,y-2,4], [x-2,y-2,5], [x-2,y-2,6], [x-2,y-2,7],
        [x-2,y-1,0], [x-2,y-1,1], [x-2,y-1,2], [x-2,y-1,3], [x-2,y-1,4], [x-2,y-1,5], [x-2,y-1,6], [x-2,y-1,7],
        [x-2,y,0], [x-2,y,1], [x-2,y,2], [x-2,y,3], [x-2,y,4], [x-2,y,5], [x-2,y,6], [x-2,y,7],
        [x-1,y-2,0], [x-1,y-2,1], [x-1,y-2,2], [x-1,y-2,3], [x-1,y-2,4], [x-1,y-2,5], [x-1,y-2,6], [x-1,y-2,7],
        [x-1,y-1,0], [x-1,y-1,1], [x-1,y-1,2], [x-1,y-1,3], [x-1,y-1,4], [x-1,y-1,5], [x-1,y-1,6], [x-1,y-1,7],
        [x-1,y,0], [x-1,y,1], [x-1,y,2], [x-1,y,3], [x-1,y,4], [x-1,y,5], [x-1,y,6], [x-1,y,7],
        [x,y-2,0], [x,y-2,1], [x,y-2,2], [x,y-2,3], [x,y-2,4], [x,y-2,5], [x,y-2,6], [x,y-2,7],
        [x,y-1,0], [x,y-1,1], [x,y-1,2], [x,y-1,3], [x,y-1,4], [x,y-1,5], [x,y-1,6], [x,y-1,7],
        [x,y,0], [x,y,1], [x,y,2], [x,y,3], [x,y,4], [x,y,5], [x,y,6], [x,y,7],
    ]
    
    coord_result = []
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 4) and (0 <= n[1] <= 4):
            coord_result.append(n)
            
    for coord in coord_result:
        result.append(dict_neurons_to_numbers[(coord[0],coord[1],coord[2])])
        #result.append(str(dict_neurons_to_numbers[(coord[0],coord[1],coord[2])]) + " - [" + str(coord[0]) + "," + str(coord[1]) + "," + str(coord[2]) + "]")
    return result

In [ ]:
get_neurons_indexes_by_position(3,3)

In [ ]:
def get_related_neurons_visible_positions(x, y):
    # Get the positions visible from the current position
    visible_coords = get_visible_coordinates_by_position(x, y)
    
    # Get the complete list of neurons related to all visible positions
    list_neurons_index = []

    for coord in visible_coords:
        neurons = get_neurons_indexes_by_position(coord[0], coord[1])
        
        for n in neurons:
            list_neurons_index.append(n)
    
    # Remove duplicates in list_neurons_index
    result = []
    
    for n in list_neurons_index:
        if n not in result:
            result.append(n)
            
    return result

In [ ]:
get_related_neurons_visible_positions(0,2)

### Auxiliary methods for Brian Simulation

In [ ]:
# Print plot for a state monitor
def plot_state_monitor(monitor, neuron):
    plot(monitor.t/ms, monitor.v[neuron]/mV)
    xlabel('Time (ms)')
    ylabel('v');
    
sns.set(style="whitegrid",font_scale=2.5, rc={'figure.figsize':(60,20)})


def seaborn_state_monitor(monitor, neuron): 
    ax = sns.lineplot(monitor.t/ms, monitor.v[neuron]/mV, linewidth=3.0)
    
    '''
    cont = STEP_TIME/ms
    for a in range(0, len(list_coordinates_optimal_path)+1):
        plt.axvline(x=cont*a, color="red")

    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
    '''
    plt.plot()

In [ ]:
### FUNCTIONS TO LOAD DATA FROM EXTERNAL FILES (TOPOLOGY, WEIGHTS, PARAMETERS...) ###

def getSynapsisDataFromFile(filename):
    synapsysData = []
    minWeight = 0
    maxWeight = 0
    
    with open(filename) as csvfile:
        csvReader = csv.reader(csvfile, delimiter=';')
        next(csvReader, None) # skip header
        
        firstRow =  next(csvReader)
        synapsysData.append([int(firstRow[0]), int(firstRow[1]), float(firstRow[2])])
        minWeight = float(firstRow[2])
        maxWeight = float(firstRow[2])
        
        for row in csvReader:
            synapsysData.append([int(row[0]), int(row[1]), float(row[2])])
            
            if minWeight > float(row[2]):
                minWeight = float(row[2])
            if maxWeight < float(row[2]):
                maxWeight = float(row[2])
    
    return synapsysData, minWeight, maxWeight

def loadIzhikevichParamI(fileName):
    with open(fileName, 'r') as reader:
        I = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(I)):
            I[i] = float(I[i])*mV/ms
        
        return I
    
def load_initial_voltages(fileName):
    with open(fileName, 'r') as reader:
        v = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(v)):
            v[i] = float(v[i])*mV
        
        return v
    
#I = loadIzhikevichParamI("paramI.txt")


def exportDataStateMonitor(dataStateMon):
    dataStoreStateMon = []
    
    # STATEMON
    # attr (v), time (0-999), neuron (0-199)
    # if execTime == 100ms, len is 1k iterations
    for time in range(0, len(dataStateMon['v'])):
        timeData = []

        for neuron in range(0, len(dataStateMon['v'][time])):
            timeData.append(dataStateMon['v'][time][neuron])
        
        dataStoreStateMon.append(timeData)

     # Result: for each instant, the V of each single neuron
    
    return dataStoreStateMon


def exportDataSpikeMonitor(dataSpikeMon):          
    # SPIKEMON
    # "t": time of each spike during the simulation
    # "i": nb of neuron that spikes, in chronological order. 
    # "count": indicates, for each neuron, the nb of spikes during the simulation

    # Result: array of 2 positions that contains:
    #     0) array where each position has the binome <time instant, nb of neuron>
    #     1) the array "count" with the nb of spikes per neuron
    #
    # Precondition: nb elements in "t" == nb elments in "i"
    
    dataStoreSpikeMon = []
    timeSpikeMon = []
    countSpikeMon = []

    for time in range(0, len(dataSpikeMon['t'])):
        # Each position: <time, nNeuron>
        timeSpikeMon.append([dataSpikeMon['t'][time], dataSpikeMon['i'][time]])
    
    for nSpikes in range(0, len(dataSpikeMon['count'])):
        # Each position: nSpikes
        countSpikeMon.append(dataSpikeMon['count'][nSpikes])
    
    dataStoreSpikeMon = [timeSpikeMon, countSpikeMon]
    
    return dataStoreSpikeMon


def exportAllDataMonitors(stateMons, spikeMons):
    # Generate data to export to CSV (suitable format for later processing)
    dataStateMon1 = exportDataStateMonitor(stateMons[0].get_states(units=False))
    dataStateMon2 = exportDataStateMonitor(stateMons[1].get_states(units=False))
    dataStateMon3 = exportDataStateMonitor(stateMons[2].get_states(units=False))

    dataSpikeMon1 = exportDataSpikeMonitor(spikeMons[0].get_states(units=False))
    dataSpikeMon2 = exportDataSpikeMonitor(spikeMons[1].get_states(units=False))
    dataSpikeMon3 = exportDataSpikeMonitor(spikeMons[2].get_states(units=False))
    
    return [dataStateMon1, dataSpikeMon1, dataStateMon2, dataSpikeMon2, dataStateMon3, dataSpikeMon3]

### FUNCTIONS TO GENERATE RANDOM NUMBER OF NEURONS ###

def generateRandomNeurons(currentNeurons):
    selectedNeurons = []
    
    # Generate random coordinates for the current number of neurons
    for currentRandom in range(0, currentNeurons):

        randNeuron = np.random.randint(0, 200)

        # Avoid duplicates
        while (randNeuron in selectedNeurons):
            randNeuron = np.random.randint(0, 200)

        # Store the selected neuron
        selectedNeurons.append(randNeuron)
        
    return selectedNeurons

    
def generatePairsRandomNeurons(currentNeurons):    
    selectedNeurons = []
    selectedPairsNeurons = []
    
    # Generate random coordinates for the current number of neurons -> pairs of neurons
    for currentRandom in range(0, currentNeurons):
        # Neuron 1 of the pair
        randNeuron1 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 1
        while (randNeuron1 in selectedNeurons):
            randNeuron1 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron1)

        # Neuron 2 of the pair
        randNeuron2 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 2
        while (randNeuron2 in selectedNeurons):
            randNeuron2 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron2)

        selectedPairsNeurons.append([randNeuron1, randNeuron2])
        
    return selectedPairsNeurons

### Load simulation parameters from files (weights, synapsis, I param...)

In [ ]:
### VARIABLES FOR THE SIMULATION OF THE ATTACKS ###

BASIC_MODEL = 0
IZHIKEVICH_MODEL = 1

# Load only once synapsis and weights
#dataSynapsisMaze_Conv1, minWeightsMaze_Conv1, maxWeightsMaze_Conv1 = getSynapsisDataFromFile("synapsysMaze-Conv1.csv")
dataSynapsisConv1_Conv2, minWeightsConv1_Conv2, maxWeightsConv1_Conv2 = getSynapsisDataFromFile("synapsysConv1-Conv2.csv")
dataSynapsisConv2_Dense, minWeightsConv2_Dense, maxWeightsConv2_Dense = getSynapsisDataFromFile("synapsysConv2-Dense.csv")

# Process the data for the simulator
#initSourceNeuronsMaze_Conv1 = []
#initTargetNeuronsMaze_Conv1 = []
initSourceNeuronsConv1_Conv2 = []
initTargetNeuronsConv1_Conv2 = []
initSourceNeuronsConv2_Dense = []
initTargetNeuronsConv2_Dense = []
initWeightsMaze_Conv1 = []
initWeightsConv1_Conv2 = []
initWeightsConv2_Dense = []

#for syn in range(0, len(dataSynapsisMaze_Conv1)):
#    initSourceNeuronsMaze_Conv1.append(dataSynapsisMaze_Conv1[syn][0])
#    initTargetNeuronsMaze_Conv1.append(dataSynapsisMaze_Conv1[syn][1])
#    initWeightsMaze_Conv1.append(dataSynapsisMaze_Conv1[syn][2])

for syn in range(0, len(dataSynapsisConv1_Conv2)):
    initSourceNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][0])
    initTargetNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][1])
    initWeightsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][2])

for syn in range(0, len(dataSynapsisConv2_Dense)):
    initSourceNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][0])
    initTargetNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][1])
    initWeightsConv2_Dense.append(dataSynapsisConv2_Dense[syn][2])


# Normalize weights for the Izhikevich model
#np_initWeightsMaze_Conv1 = np.array(initWeightsMaze_Conv1)
np_initWeightsConv1_Conv2 = np.array(initWeightsConv1_Conv2)
np_initWeightsConv2_Dense = np.array(initWeightsConv2_Dense)

#norm_initWeightsMaze_Conv1 = np.interp(np_initWeightsMaze_Conv1, (np_initWeightsMaze_Conv1.min(), np_initWeightsMaze_Conv1.max()), (min_range_V, max_range_V))
norm_initWeightsConv1_Conv2 = np.interp(np_initWeightsConv1_Conv2, (np_initWeightsConv1_Conv2.min(), np_initWeightsConv1_Conv2.max()), (min_range_V, max_range_V))
norm_initWeightsConv2_Dense = np.interp(np_initWeightsConv2_Dense, (np_initWeightsConv2_Dense.min(), np_initWeightsConv2_Dense.max()), (min_range_V, max_range_V))

N_NEURONS_MAX = 100
N_NEURONS_MIN = 1

attacks_dict = {
    'FLO': ['Flooding', N_NEURONS_MAX, [0.25, 0.5, 0.75, 1.0]], # Stim multiple neurons per t.u.
    'JAM': ['Jamming', N_NEURONS_MAX, [-1.0]], # Inhibit multiple neurons per t.u.
    'SCA': ['PortScanning', N_NEURONS_MIN, [0.25, 0.5, 0.75, 1.0]],  # Stim 1 neuron per t.u.
    'FOR': ['SelectiveForwarding', N_NEURONS_MIN, [-1.0]], # Inhibit 1 neuron per t.u.
    'SPO': ['Spoofing', int(N_NEURONS_MAX/2)], # the attack selects pairs of neurons -> double of this number
    'SYB': ['Sybil', N_NEURONS_MAX],
}

maze =  np.array([
   [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  0.,  1.,  0.], 
    [ 0.,  0.,  0.,  1.,  1.,  1.,  0.],
    [ 1.,  1.,  1.,  1.,  0.,  0.,  1.],
    [ 1.,  0.,  0.,  0.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.]
])


# Load initial voltages for all simulation (previously generated random values)
v_initial = load_initial_voltages("initial_voltage.txt")

### Normal simulation (without attacks)

In [ ]:
def spontaneous_simulation(simulationDuration, paramI):
    start_scope()
    
    #defaultclock.dt = 0.1*ms
    
    # Equations of the Izhikevich neuron model
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u + I : volt
    du/dt = a*(b*v-u) : volt/second
    I : volt/second
    a : Hz
    b : Hz
    c : volt
    d : volt/second
    neuronCounter : 1
    positionCounter : 1
    isFirstTime : 1
    timeCounter : second
    '''

    # Reset of Izhikevich model
    reset ='''
    v = c
    u += d
    '''

    thresholdValue = 30
    resetValue = -65

    # Definition of the 1st layer of the CNN
    G = NeuronGroup(276, eqs, threshold='v >= thresholdValue*mV', reset=reset, method='euler')
    # Initialise variables of all neurons (typical values)

    #G.v = resetValue*mV
    G.v = v_initial
    
    G.u = -13*mV/ms # b*v -> 0.2*-65 = -13
    G.a = 0.02/ms
    G.b = 0.2/ms
    G.c = resetValue*mV
    G.d = 8*mV/ms
    
    #G.I = 10*mV/ms
    G.I = paramI*mV/ms
    #G.I = loadIzhikevichParamI("paramI.txt")
    
    # Create subgroups
    layerConv1 = G[:200]
    layerConv2 = G[200:272]
    layerDense = G[272:276]

    layerConv1.neuronCounter[0] = 0
    layerConv1.positionCounter[0] = 0
    layerConv1.timeCounter[0] = 0*ms
    layerConv1.isFirstTime[0] = 1
        
    # Monitors G
    stateMonGlobal = StateMonitor(G, 'v', record=True)
    #stateMon1 = StateMonitor(layerConv1, 'v', record=True)
    #stateMon2 = StateMonitor(layerConv2, 'v', record=True)
    #stateMon3 = StateMonitor(layerDense, 'v', record=True)

    spikeMonGlobal = SpikeMonitor(G)
    #spikeMon1 = SpikeMonitor(layerConv1)
    #spikeMon2 = SpikeMonitor(layerConv2)
    #spikeMon3 = SpikeMonitor(layerDense)
    
    # Synapsis definition
    synConv1_Conv2 = Synapses(layerConv1, layerConv2, 'w : volt', on_pre='v_post += w')
    synConv2_Dense = Synapses(layerConv2, layerDense, 'w : volt', on_pre='v_post += w')
    
    # Connect synapsis
    synConv1_Conv2.connect(i=initSourceNeuronsConv1_Conv2, j=initTargetNeuronsConv1_Conv2)
    synConv2_Dense.connect(i=initSourceNeuronsConv2_Dense, j=initTargetNeuronsConv2_Dense)
    
    synConv1_Conv2.w = norm_initWeightsConv1_Conv2*mV
    synConv2_Dense.w = norm_initWeightsConv2_Dense*mV
    
    '''
    @network_operation(dt=STEP_TIME)
    def periodicFunction():
        positionCounter = int(layerConv1.positionCounter[0])
        #print("current_counter: ", positionCounter)
        
        if layerConv1.isFirstTime[0] == 1:
            layerConv1.isFirstTime[0] = 0
        else:
            if positionCounter < len(list_coordinates_optimal_path):
                # Reset all neurons to default I value            
                #G.I = loadIzhikevichParamI("paramI.txt")   # Random values for all neurons
                G.I = 10*mV/ms
                
                # Get the coordinates of the current position
                coord = list_coordinates_optimal_path[positionCounter]

                # Update I value only for the neurons related to the visible positions from the current one
                list_neurons_index = get_related_neurons_visible_positions(coord[0], coord[1])

                for neuron in list_neurons_index:
                    G.I[neuron] = 15*mV/ms

                # Update the counter of the current position over the maze (for next iteration)
                layerConv1.positionCounter[0] += 1
    '''
    run(simulationDuration)
    
    # stateMons, spikeMons
    #return [[spikeMon1, spikeMon2, spikeMon3, spikeMonGlobal], [stateMon1, stateMon2, stateMon3, stateMonGlobal]]
    return [spikeMonGlobal, stateMonGlobal]

#spikeMons_init, stateMons_init = modelsWithoutAttacks(IZHIKEVICH_MODEL, SIMULATION_TIME)

# Export to CSV
#dump_simulation_data_to_csv("initial_state", "1", "X", "X", "0", "0", spikeMons_init[3])   

In [ ]:
spikeMons_10, stateMons_10 = spontaneous_simulation(200*ms, 10)
spikeMons_15, stateMons_15 = spontaneous_simulation(200*ms, 15)

In [ ]:
plt.rcParams['axes.edgecolor'] = "black" #set the value globally

fig,axs = plt.subplots(nrows=2, ncols=1, sharey='row', sharex='col', figsize=(80,40))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)

label_size = 100
ticks_size = 90
legend_size = 90
line_size = 20

neuron = 0

sns.lineplot(stateMons_10.t/ms, stateMons_10.v[neuron]/mV, linewidth=line_size, color="purple", label="Non intervening neuron", ax=axs[0])
sns.lineplot(stateMons_15.t/ms, stateMons_15.v[neuron]/mV, linewidth=line_size, color="green", label="Intervening neuron", ax=axs[1])

axs[0].set_ylabel("Voltage (mV)", fontsize=label_size, fontweight="bold")
axs[0].tick_params(labelsize=ticks_size)

axs[1].set_ylabel("Voltage (mV)", fontsize=label_size, fontweight="bold")
axs[1].set_xlabel("Time duration (ms)", fontsize=label_size, fontweight="bold")
axs[1].tick_params(labelsize=ticks_size)

leg = axs[0].legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)        
leg.get_frame().set_edgecolor('black')

leg = axs[1].legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)        
leg.get_frame().set_edgecolor('black')

fig.savefig("differences_I.pdf")
plt.close()